In [13]:
import numpy as np

from fashion_mnist_cnn_v2 import *

In [14]:
def selection(pop, fitness, num_parents):
    # We select the ones with the highest fitness
    sorted_indecies = np.argsort(fitness)[::-1] # we reverse array, because we want ascending
    top_indecies = sorted_indecies[:num_parents]
    parents = pop[top_indecies]
    return parents

def crossover(parents, offspring_size):
    offspring = np.empty((offspring_size, parents.shape[1]))
    # The point at which crossover takes place between two parents
    crossover_part = int(parents.shape[1]/2)
    # Define all the offsprings
    for k in range(0, offspring_size, 2):
        # we assign random from the two parents
        random_choice = np.random.choice(parents.shape[1], crossover_part, replace=False)
        gene_selection = np.zeros(parents.shape[1])
        gene_selection[random_choice] = 1
        # Index of the first parent to mate.
        parent1_idx = k%parents.shape[0]
        # Index of the second parent to mate.
        parent2_idx = (k+1)%parents.shape[0]
        # Assign the correct parts of the parents to the offspring
        offspring[k, np.where(gene_selection == 0)[0]] = parents[parent1_idx, np.where(gene_selection == 0)[0]]
        offspring[k, np.where(gene_selection == 1)[0]] = parents[parent2_idx, np.where(gene_selection == 1)[0]]
        offspring[k+1, np.where(gene_selection == 1)[0]] = parents[parent1_idx, np.where(gene_selection == 1)[0]]
        offspring[k+1, np.where(gene_selection == 0)[0]] = parents[parent2_idx, np.where(gene_selection == 0)[0]]
    return offspring

def mutation(offspring_crossover, r_mut=0.2, v_mut=0.2):
    # Do if mutation stikes, must be changed if number of options exceed 2
    for idx in range(offspring_crossover.shape[0]):
        for gene_idx in range(offspring_crossover.shape[1]): 
            # Flip gene if mutation happens
            if (np.random.rand() < r_mut): 
                new_value = offspring_crossover[idx, gene_idx] + (np.random.rand()*2-1)*v_mut
                if (new_value < 0): 
                    new_value = 0
                elif (new_value >= 1): 
                    new_value = 0.99
                offspring_crossover[idx, gene_idx] = new_value 
                #offspring_crossover[idx, gene_idx] = 1-offspring_crossover[idx, gene_idx]# bit flip
    return offspring_crossover

In [8]:
"""
The target is to maximize the accuracy of the cnn-mnist model:
    We start by having a few decisions related to the model. 
    // Model hyper-parameters
    x1 = 32-128 filters in each CNN layer
    x2 = 50-100 neurons in first dense layer
    x3 = no dropout, 0.1 dropout, 0.2 dropout
    x4 = learning rate 0.005 or 0.05 
    x5 = kernel size cnn 2-4
    
    // Data augmentation hyper-parameters
    x6 = rotation_range: 0-30
    x7 = width_shift_range: 0.0-0.3,
    x8 = height_shift_range: 0.0-0.3
    x9 = horizontal_flip: 0 or 1
    
    We will use a genetic algorithm to find the best combinations of these hyperparameters. 
    The fitness-function is based on model accuracy on test-dataset. 
"""
num_weights = 9

# solutions per populations and number of mating parents
sol_per_pop = 6
num_parents = 3

In [9]:
# Defining the population size.
pop_size = (sol_per_pop,num_weights) # The population will have sol_per_pop chromosome where each chromosome has num_weights genes.
#Creating the initial population.
new_population = np.random.uniform(low=0, high=1, size=pop_size)
print(new_population)

[[0.74037167 0.4099496  0.10700141 0.82154997 0.12580744 0.82534966
  0.2089763  0.02643974 0.4974076 ]
 [0.59555797 0.2135547  0.30959775 0.22033295 0.05368361 0.54768959
  0.29424988 0.47143987 0.85986344]
 [0.13947814 0.24184284 0.73953399 0.37256134 0.26417414 0.56936304
  0.21418519 0.11457757 0.6163004 ]
 [0.88537386 0.98025384 0.29799092 0.55029356 0.05576576 0.44821935
  0.49676298 0.66463962 0.30668779]
 [0.08431895 0.55684617 0.09736169 0.41034473 0.54115522 0.3095216
  0.62258839 0.54324093 0.83692658]
 [0.05715461 0.46610457 0.29783137 0.64388566 0.73804022 0.26118112
  0.91260881 0.26015466 0.132359  ]]


In [12]:
best_outputs = []
num_generations = 10
generation_results = []
checkpoint_file = "fit_pop.json"

In [11]:
for generation in range(num_generations):
    print("Generation : ", generation)
    # Measuring the fitness of each chromosome in the population.
    fitness = evaluate_model_ga(new_population)
    print("Fitness")
    print(fitness)
    
    # save the results into a list and do intemediate saving
    generation_results.append({"fitness": fitness, "population": new_popolation})
    with open(checkpoint_file, "w") as f: 
        json.dump(generation_results, f)
    
    # Find best result from gen x
    generation_best = (np.max(fitness), new_population[np.where(fitness == np.max(fitness))[0]])
    best_outputs.append(generation_best)
    # The best result in the current iteration.
    print("Best result : ", generation_best)
    
    # Selecting the best parents in the population for mating.
    parents = selection(new_population, fitness, num_parents)
    print("Parents")
    print(parents)

    # Generating next generation using crossover.
    offspring_crossover = crossover(parents, sol_per_pop)
    print("Crossover")
    print(offspring_crossover)

    # Adding some variations to the offspring using mutation.
    offspring_mutation = mutation(offspring_crossover, r_mut=0.2)
    print("Mutation")
    print(offspring_mutation)

    # Creating the new population based on the parents and offspring.
    new_population = offspring_mutation

Generation :  0
Epoch 1/30
1500/1500 [==============================] - 26s 17ms/step - loss: 0.8495 - accuracy: 0.6824 - val_loss: 0.5257 - val_accuracy: 0.8069
Epoch 2/30
1500/1500 [==============================] - 25s 16ms/step - loss: 0.4963 - accuracy: 0.8131 - val_loss: 0.4187 - val_accuracy: 0.8451
Epoch 3/30
1500/1500 [==============================] - 25s 17ms/step - loss: 0.4399 - accuracy: 0.8357 - val_loss: 0.3976 - val_accuracy: 0.8507
Epoch 4/30
1500/1500 [==============================] - 25s 17ms/step - loss: 0.4162 - accuracy: 0.8459 - val_loss: 0.4158 - val_accuracy: 0.8469
Epoch 5/30
1500/1500 [==============================] - 25s 17ms/step - loss: 0.4027 - accuracy: 0.8509 - val_loss: 0.3980 - val_accuracy: 0.8496
Epoch 6/30
1500/1500 [==============================] - 25s 17ms/step - loss: 0.3995 - accuracy: 0.8508 - val_loss: 0.3800 - val_accuracy: 0.8651
Epoch 7/30
1500/1500 [==============================] - 25s 17ms/step - loss: 0.3832 - accuracy: 0.8584 - va

KeyboardInterrupt: 